In [4]:
import pandas as pd
import numpy as np
import pickle
import plotly.graph_objects as go

from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

#importing the data
x = pd.read_csv('x.csv').drop('Unnamed: 0', axis=1)
y = pd.read_csv('y.csv').drop('Unnamed: 0', axis=1)

#storing the details
x_cols = list(x.columns)
titles = list(y.columns)

x_cols

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 'posted_at',
 'post_type_link',
 'post_type_music',
 'post_type_note',
 'post_type_offer',
 'post_type_photo',
 'post_type_status',
 'post_type_video',
 'status_type_added_video',
 'status_type_created_event',
 'status_type_created_note',
 'status_type_mobile_status_update',
 'status_type_published_story',
 'status_type_shared_story',
 'source_bbc',
 'source_cbs',
 'source_cnn',
 'source_lat',
 'source_nbc',
 'source_nyt',
 'source_wsj']

In [5]:
#Specific cols
num_cols = ['posted_at', '0', '1', '2', '3', '4', '5', '6']
cols = [col for col in x_cols if col not in num_cols]

#preprocessing the values
preprocess = ColumnTransformer([
    (
        'numeric',
        MinMaxScaler(),
        num_cols
    )
], remainder='passthrough')

x = preprocess.fit_transform(x)

#Getting the final order of columns right...
final_cols = []
final_cols.extend(num_cols)
final_cols.extend(cols)
# print(final_cols)

x_true = pd.DataFrame(x, columns = final_cols)

#Train,test split
x_train,x_val,y_train,y_val = train_test_split(x, np.array(y.values), test_size=0.1, shuffle=True, random_state=35)

In [6]:
def save_model(pkl_model,name):
    pkl_dir = './models/ml/'
    pkl_filename =pkl_dir+ name+'.pkl'
    with open(pkl_filename, 'wb') as file:
        pickle.dump(pkl_model, file)

# SVM - regression chain vs Multioutput Regressor

**Multi-Output Regressor**: The regression in which all the output variables are predicted at once.
**Regression Chain**: The data is used to predict output target-1, which is then combined with the data to predict output variable 2 and so on.

In [7]:
svr = SVR()
multi_svr = MultiOutputRegressor(svr)
multi_svr.fit(X=x_train,y=y_train)

In [ ]:
pred = multi_svr.predict(x_val)
save_model(multi_svr, 'video_svr_multi')

In [ ]:
multi_svr_scores = []

for i in range(9):
    mse = mean_squared_error(y_true=y_val[:,i],y_pred=pred[:,i])
    mae = mean_absolute_error(y_pred=pred[:,i],y_true=y_val[:,i])
    print(f'Mean Squared Error for parameter - {i}: {mse}')
    print(f'Mean Absolute Error for parameter - {i}: {mae}')
    multi_svr_scores.append([mse,mae])
    print('\n')

multi_svr_scores.append([mean_squared_error(y_true=y_val,y_pred=pred),mean_absolute_error(y_true=y_val,y_pred=pred)])


Mean Squared Error for parameter - 0: 13121.803774051064
Mean Absolute Error for parameter - 0: 36.7706717565298


Mean Squared Error for parameter - 1: 1589.9283488137753
Mean Absolute Error for parameter - 1: 11.22586518329064


Mean Squared Error for parameter - 2: 3250.5931616135686
Mean Absolute Error for parameter - 2: 14.353393485089178


Mean Squared Error for parameter - 3: 2159.8565204843535
Mean Absolute Error for parameter - 3: 10.907644190322943


Mean Squared Error for parameter - 4: 15.980091015816958
Mean Absolute Error for parameter - 4: 1.3414989748567232


Mean Squared Error for parameter - 5: 1.3742165474556884
Mean Absolute Error for parameter - 5: 0.4271668990639327


Mean Squared Error for parameter - 6: 386.92893930184283
Mean Absolute Error for parameter - 6: 2.5621464270315366


Mean Squared Error for parameter - 7: 39.790746666915034
Mean Absolute Error for parameter - 7: 1.0789455284992475


Mean Squared Error for parameter - 8: 16.810255351098803
Mean Absol

In [ ]:
chain_poisson = RegressorChain(svr, order=[0,1,2,3,4,5,6,7,8])
chain_poisson.fit(x_train,y_train)

RegressorChain(base_estimator=SVR(), order=[0, 1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:
pred = chain_poisson.predict(x_val)
save_model(chain_poisson, 'video_svr_chain')
pred

array([[6.32257484e+00, 4.45415297e+00, 3.87495026e+00, ...,
        8.63858597e-02, 1.02710709e-01, 9.69090138e-02],
       [7.40741366e+00, 4.49613368e+00, 3.92254688e+00, ...,
        8.59290467e-02, 1.02467136e-01, 9.66664391e-02],
       [8.13379018e+00, 4.52425113e+00, 3.95440856e+00, ...,
        8.56220914e-02, 1.02293231e-01, 9.65049134e-02],
       ...,
       [3.49555753e+01, 5.57029957e+00, 5.14052879e+00, ...,
        7.48499238e-02, 9.62023650e-02, 9.04704328e-02],
       [2.96390170e+01, 5.36207277e+00, 4.90415513e+00, ...,
        7.69481904e-02, 9.73343139e-02, 9.17939891e-02],
       [7.15802070e+01, 7.01131594e+00, 6.77180306e+00, ...,
        6.18337121e-02, 8.86673604e-02, 8.23727925e-02]])

In [ ]:
chain_svr_scores = []

for i in range(9):
    mse = mean_squared_error(y_true=y_val[:,i],y_pred=pred[:,i])
    mae = mean_absolute_error(y_pred=pred[:,i],y_true=y_val[:,i])
    print(f'Mean Squared Error for parameter - {i}: {mse}')
    print(f'Mean Absolute Error for parameter - {i}: {mae}')
    chain_svr_scores.append([mse,mae])
    print('\n')

chain_svr_scores.append([mean_squared_error(y_true=y_val,y_pred=pred),mean_absolute_error(y_true=y_val,y_pred=pred)])


Mean Squared Error for parameter - 0: 13121.803774051064
Mean Absolute Error for parameter - 0: 36.7706717565298


Mean Squared Error for parameter - 1: 1919.9570031281005
Mean Absolute Error for parameter - 1: 12.772797241651457


Mean Squared Error for parameter - 2: 3570.5180386550683
Mean Absolute Error for parameter - 2: 15.510604800614026


Mean Squared Error for parameter - 3: 2537.198672270509
Mean Absolute Error for parameter - 3: 12.293494190573618


Mean Squared Error for parameter - 4: 20.3990485140151
Mean Absolute Error for parameter - 4: 1.4620272618028034


Mean Squared Error for parameter - 5: 1.409946375784454
Mean Absolute Error for parameter - 5: 0.4171743202786188


Mean Squared Error for parameter - 6: 388.8374768790617
Mean Absolute Error for parameter - 6: 2.5674694279674357


Mean Squared Error for parameter - 7: 39.82023693768767
Mean Absolute Error for parameter - 7: 1.077099491100074


Mean Squared Error for parameter - 8: 19.952393047466124
Mean Absolute Er

## Decision Tree

In [ ]:
tree = DecisionTreeRegressor()
tree.fit(x_train,y_train)
pred = tree.predict(x_val)
save_model(tree, 'video_tree')
pred

array([[  7.,   1.,   1., ...,   0.,   1.,   0.],
       [  3.,   6.,   1., ...,   0.,   3.,   0.],
       [ 10.,   1.,   3., ...,   0.,   0.,   0.],
       ...,
       [416.,  63., 108., ...,   0.,   0.,   0.],
       [ 18.,   4.,   3., ...,   0.,   0.,   0.],
       [304.,  25., 166., ...,  37.,   0.,   0.]])

In [ ]:
decision_tree_scores = []

for i in range(9):
    mse = mean_squared_error(y_true=y_val[:,i],y_pred=pred[:,i])
    mae = mean_absolute_error(y_pred=pred[:,i],y_true=y_val[:,i])
    print(f'Mean Squared Error for parameter - {i}: {mse}')
    print(f'Mean Absolute Error for parameter - {i}: {mae}')
    decision_tree_scores.append([mse,mae])
    print('\n')

decision_tree_scores.append([mean_squared_error(y_true=y_val,y_pred=pred),mean_absolute_error(y_true=y_val,y_pred=pred)])


Mean Squared Error for parameter - 0: 8472.11320754717
Mean Absolute Error for parameter - 0: 30.32704402515723


Mean Squared Error for parameter - 1: 895.6477987421383
Mean Absolute Error for parameter - 1: 11.452830188679245


Mean Squared Error for parameter - 2: 4979.707547169812
Mean Absolute Error for parameter - 2: 19.09748427672956


Mean Squared Error for parameter - 3: 1560.0157232704403
Mean Absolute Error for parameter - 3: 11.10377358490566


Mean Squared Error for parameter - 4: 24.443396226415093
Mean Absolute Error for parameter - 4: 1.7830188679245282


Mean Squared Error for parameter - 5: 7.30503144654088
Mean Absolute Error for parameter - 5: 0.7578616352201258


Mean Squared Error for parameter - 6: 694.2767295597484
Mean Absolute Error for parameter - 6: 3.880503144654088


Mean Squared Error for parameter - 7: 47.198113207547166
Mean Absolute Error for parameter - 7: 1.5188679245283019


Mean Squared Error for parameter - 8: 7.79874213836478
Mean Absolute Error 

In [ ]:
#Scores for estimation
df = pd.DataFrame(multi_svr_scores, columns=['Multi-SVR: MSE', 'Multi-SVR: MAE'])

df = pd.concat([df, pd.DataFrame(chain_svr_scores, columns=['Chain-SVR: MSE', 'Chain-SVR: MAE'])],
          axis=1)

df = pd.concat([df,pd.DataFrame(decision_tree_scores, columns=['D-Tree: MSE', 'D-Tree: MAE'])],
          axis=1)

In [ ]:
df

,Multi-SVR: MSE,Multi-SVR: MAE,Chain-SVR: MSE,Chain-SVR: MAE,D-Tree: MSE,D-Tree: MAE
0,13121.803774,36.770672,13121.803774,36.770672,8472.113208,30.327044
1,1589.928349,11.225865,1919.957003,12.772797,895.647799,11.452830
2,3250.593162,14.353393,3570.518039,15.510605,4979.707547,19.097484
3,2159.856520,10.907644,2537.198672,12.293494,1560.015723,11.103774
4,15.980091,1.341499,20.399049,1.462027,24.443396,1.783019
5,1.374217,0.427167,1.409946,0.417174,7.305031,0.757862
6,386.928939,2.562146,388.837477,2.567469,694.276730,3.880503
7,39.790747,1.078946,39.820237,1.077099,47.198113,1.518868
8,16.810255,0.480872,19.952393,0.558123,7.798742,0.440252
9,2287.007339,8.794245,2402.210732,9.269940,1854.278477,8.929071


## Linear Regression

In [ ]:
linear = LinearRegression()
linear.fit(x_train,y_train)
pred = linear.predict(x_val)
pred

array([[-9.56821302e+00,  3.20900352e+00,  3.32703338e+00, ...,
        -1.45885838e-01,  2.28679444e-01, -1.53384458e-01],
       [-1.18457008e+02, -2.06914232e-01, -5.12208525e+01, ...,
         5.03040765e+00,  1.42276419e+00, -8.32764361e+00],
       [ 1.43561793e+02,  2.44940691e+01,  1.80297631e+02, ...,
        -1.93570200e+00,  5.54413667e-01, -3.01470343e-01],
       ...,
       [ 2.05573664e+02,  2.20268574e+01,  1.06714565e+02, ...,
        -2.66493515e+00, -1.69198436e+00,  2.88248904e+00],
       [-4.92620568e+01,  8.38921177e+00,  2.33073991e+01, ...,
         9.38942721e-01,  2.59106046e-01, -1.52745359e+01],
       [-2.21475816e+02, -3.60579099e+01,  3.51922530e+01, ...,
         7.49129625e+01,  6.48462742e+00, -1.28385715e+02]])

In [ ]:

save_model(linear, 'video_linear')

linear_regression_scores = []

for i in range(9):
    mse = mean_squared_error(y_true=y_val[:,i],y_pred=pred[:,i])
    mae = mean_absolute_error(y_pred=pred[:,i],y_true=y_val[:,i])
    print(f'Mean Squared Error for parameter - {i}: {mse}')
    print(f'Mean Absolute Error for parameter - {i}: {mae}')
    linear_regression_scores.append([mse,mae])
    print('\n')

linear_regression_scores.append([mean_squared_error(y_true=y_val,y_pred=pred),mean_absolute_error(y_true=y_val,y_pred=pred)])
df = pd.concat([df,pd.DataFrame(linear_regression_scores, columns=['Linear: MSE', 'Linear: MAE'])],
          axis=1)

Mean Squared Error for parameter - 0: 137887.89459492583
Mean Absolute Error for parameter - 0: 151.86694069303547


Mean Squared Error for parameter - 1: 1675.9918888636932
Mean Absolute Error for parameter - 1: 17.71316560449715


Mean Squared Error for parameter - 2: 45678.80713912998
Mean Absolute Error for parameter - 2: 97.20915249628175


Mean Squared Error for parameter - 3: 43633.89957318244
Mean Absolute Error for parameter - 3: 109.71757413132994


Mean Squared Error for parameter - 4: 985.1773122069492
Mean Absolute Error for parameter - 4: 19.36177120701737


Mean Squared Error for parameter - 5: 1.9638512177252772
Mean Absolute Error for parameter - 5: 0.836330678302422


Mean Squared Error for parameter - 6: 340.7716947410513
Mean Absolute Error for parameter - 6: 7.4312926612705965


Mean Squared Error for parameter - 7: 45.498595672708895
Mean Absolute Error for parameter - 7: 2.7301650791253884


Mean Squared Error for parameter - 8: 1096.1119440709929
Mean Absolute E

## Random Forest

In [ ]:
forest = RandomForestRegressor()
multi_forest = MultiOutputRegressor(forest)
multi_forest.fit(x_train,y_train)
pred = multi_forest.predict(x_val)

save_model(multi_forest, 'video_forest_multi')

In [ ]:
random_forest_scores = []

for i in range(9):
    mse = mean_squared_error(y_true=y_val[:,i],y_pred=pred[:,i])
    mae = mean_absolute_error(y_pred=pred[:,i],y_true=y_val[:,i])
    print(f'Mean Squared Error for parameter - {i}: {mse}')
    print(f'Mean Absolute Error for parameter - {i}: {mae}')
    random_forest_scores.append([mse,mae])
    print('\n')

random_forest_scores.append([mean_squared_error(y_true=y_val,y_pred=pred),mean_absolute_error(y_true=y_val,y_pred=pred)])
df = pd.concat([df,pd.DataFrame(random_forest_scores, columns=['Random Forest: MSE', 'Random Forest: MAE'])],
          axis=1)

Mean Squared Error for parameter - 0: 6682.866818238994
Mean Absolute Error for parameter - 0: 25.432830188679244


Mean Squared Error for parameter - 1: 955.0343556603773
Mean Absolute Error for parameter - 1: 10.305377358490565


Mean Squared Error for parameter - 2: 1815.2711635220126
Mean Absolute Error for parameter - 2: 13.433710691823899


Mean Squared Error for parameter - 3: 957.4084625786164
Mean Absolute Error for parameter - 3: 8.128081761006289


Mean Squared Error for parameter - 4: 12.87527704402516
Mean Absolute Error for parameter - 4: 1.504559748427673


Mean Squared Error for parameter - 5: 32.21466603773586
Mean Absolute Error for parameter - 5: 1.1476729559748426


Mean Squared Error for parameter - 6: 271.88947578616353
Mean Absolute Error for parameter - 6: 3.3905974842767295


Mean Squared Error for parameter - 7: 43.34776855345911
Mean Absolute Error for parameter - 7: 1.3451572327044026


Mean Squared Error for parameter - 8: 23.517117610062893
Mean Absolute E

## Random Forest - Poisson Criterion

In [ ]:
forest = RandomForestRegressor(criterion='poisson')
multi_forest = MultiOutputRegressor(forest)
multi_forest.fit(x_train,y_train)
pred = multi_forest.predict(x_val)

save_model(multi_forest, 'video_forest_multi')

In [ ]:
random_forest_scores = []

for i in range(9):
    mse = mean_squared_error(y_true=y_val[:,i],y_pred=pred[:,i])
    mae = mean_absolute_error(y_pred=pred[:,i],y_true=y_val[:,i])
    print(f'Mean Squared Error for parameter - {i}: {mse}')
    print(f'Mean Absolute Error for parameter - {i}: {mae}')
    random_forest_scores.append([mse,mae])
    print('\n')

random_forest_scores.append([mean_squared_error(y_true=y_val,y_pred=pred),mean_absolute_error(y_true=y_val,y_pred=pred)])
df = pd.concat([df,pd.DataFrame(random_forest_scores, columns=['Random Forest_Poisson: MSE', 'Random Forest_Poisson: MAE'])],
          axis=1)

Mean Squared Error for parameter - 0: 7057.845247965239
Mean Absolute Error for parameter - 0: 25.83069510582011


Mean Squared Error for parameter - 1: 1077.798219353892
Mean Absolute Error for parameter - 1: 10.837815501676767


Mean Squared Error for parameter - 2: 1452.4595510153874
Mean Absolute Error for parameter - 2: 11.807607726849321


Mean Squared Error for parameter - 3: 976.866708283015
Mean Absolute Error for parameter - 3: 8.278825717022128


Mean Squared Error for parameter - 4: 20.194960667007763
Mean Absolute Error for parameter - 4: 1.5673460751673824


Mean Squared Error for parameter - 5: 6.881381007520304
Mean Absolute Error for parameter - 5: 0.8837655862670044


Mean Squared Error for parameter - 6: 264.8351408618017
Mean Absolute Error for parameter - 6: 3.333346072423179


Mean Squared Error for parameter - 7: 34.120407819801486
Mean Absolute Error for parameter - 7: 1.3908070822587442


Mean Squared Error for parameter - 8: 47.994098526994954
Mean Absolute Er

## Perceptron Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

2022-11-11 01:49:36.586333: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 01:49:36.704050: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2022-11-11 01:49:36.704068: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-11 01:49:36.726796: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 01:49:37.330031: W tensorflow/

In [ ]:
model = Sequential(
    [
        Dense(128, input_dim=26, activation='relu', kernel_initializer='normal'),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_initializer='normal'),
        Dropout(0.3),
        Dense(32, activation='relu', kernel_initializer='normal'),
        Dense(9, kernel_initializer='normal')
    ]
)

model.compile(loss='mse', optimizer='adam', metrics=[ tf.keras.metrics.MeanSquaredError()])

2022-11-11 01:49:38.016506: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 01:49:38.017042: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2022-11-11 01:49:38.017099: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2022-11-11 01:49:38.017152: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2022-11-11 01

In [ ]:
x_train.shape

(2855, 26)

In [ ]:
model.fit(x_train,y_train, epochs=100, batch_size=32)

Epoch 1/100
90/90 [==============================] - 1s 1ms/step - loss: 991375.3125 - mean_squared_error: 991375.3125
Epoch 2/100
90/90 [==============================] - 0s 1ms/step - loss: 988827.8125 - mean_squared_error: 988827.8125
Epoch 3/100
90/90 [==============================] - 0s 1ms/step - loss: 979043.0625 - mean_squared_error: 979043.0625
Epoch 4/100
90/90 [==============================] - 0s 1ms/step - loss: 961066.3125 - mean_squared_error: 961066.3125
Epoch 5/100
90/90 [==============================] - 0s 1ms/step - loss: 932682.4375 - mean_squared_error: 932682.4375
Epoch 6/100
90/90 [==============================] - 0s 1ms/step - loss: 873234.5000 - mean_squared_error: 873234.5000
Epoch 7/100
90/90 [==============================] - 0s 1ms/step - loss: 792711.1875 - mean_squared_error: 792711.1875
Epoch 8/100
90/90 [==============================] - 0s 1ms/step - loss: 688439.3750 - mean_squared_error: 688439.3750
Epoch 9/100
90/90 [=============================

In [ ]:
pred = model.predict(x_val)
save_model(model.get_weights(), 'video_mlp')
mean_squared_error(y_val,pred)

10/10 [==============================] - 0s 928us/step


2416.1901043455414

In [ ]:
perceptron_scores = []

for i in range(9):
    mse = mean_squared_error(y_true=y_val[:,i],y_pred=pred[:,i])
    mae = mean_absolute_error(y_pred=pred[:,i],y_true=y_val[:,i])
    print(f'Mean Squared Error for parameter - {i}: {mse}')
    print(f'Mean Absolute Error for parameter - {i}: {mae}')
    perceptron_scores.append([mse,mae])
    print('\n')

perceptron_scores.append([mean_squared_error(y_true=y_val,y_pred=pred),mean_absolute_error(y_true=y_val,y_pred=pred)])
df = pd.concat([df,pd.DataFrame(perceptron_scores, columns=['Perceptron: MSE', 'Perceptron: MAE'])],
          axis=1)

Mean Squared Error for parameter - 0: 13760.031717079832
Mean Absolute Error for parameter - 0: 47.50402707273855


Mean Squared Error for parameter - 1: 1759.8144912037203
Mean Absolute Error for parameter - 1: 17.12961823985262


Mean Squared Error for parameter - 2: 3356.235572961296
Mean Absolute Error for parameter - 2: 19.35009704445893


Mean Squared Error for parameter - 3: 2404.9846436029297
Mean Absolute Error for parameter - 3: 16.92608392613489


Mean Squared Error for parameter - 4: 18.76153368982258
Mean Absolute Error for parameter - 4: 2.186689617873738


Mean Squared Error for parameter - 5: 1.536832167976191
Mean Absolute Error for parameter - 5: 0.8103890996309197


Mean Squared Error for parameter - 6: 385.9880478768148
Mean Absolute Error for parameter - 6: 5.911091552590424


Mean Squared Error for parameter - 7: 38.83344892840694
Mean Absolute Error for parameter - 7: 1.533059389898612


Mean Squared Error for parameter - 8: 19.524651599073497
Mean Absolute Error

# Observing the results

In [ ]:
mae_cols = [col for col in df.columns if 'MAE' in col]
mse_cols = [col for col in df.columns if 'MSE' in col]

In [ ]:
df[mse_cols]

,Multi-SVR: MSE,Chain-SVR: MSE,D-Tree: MSE,Linear: MSE,Random Forest: MSE,Random Forest_Poisson: MSE,Perceptron: MSE
0,13121.803774,13121.803774,8472.113208,137887.894595,6682.866818,7057.845248,13760.031717
1,1589.928349,1919.957003,895.647799,1675.991889,955.034356,1077.798219,1759.814491
2,3250.593162,3570.518039,4979.707547,45678.807139,1815.271164,1452.459551,3356.235573
3,2159.856520,2537.198672,1560.015723,43633.899573,957.408463,976.866708,2404.984644
4,15.980091,20.399049,24.443396,985.177312,12.875277,20.194961,18.761534
5,1.374217,1.409946,7.305031,1.963851,32.214666,6.881381,1.536832
6,386.928939,388.837477,694.276730,340.771695,271.889476,264.835141,385.988048
7,39.790747,39.820237,47.198113,45.498596,43.347769,34.120408,38.833449
8,16.810255,19.952393,7.798742,1096.111944,23.517118,47.994099,19.524652
9,2287.007339,2402.210732,1854.278477,25705.124066,1199.380567,1215.443968,2416.190104


In [ ]:
titles

['Likes',
 'Comments',
 'Shares',
 'Love',
 'Wow',
 'Haha',
 'Sad',
 'Angry',
 'Care',
 'Overall']

In [ ]:
titles.extend(["Overall"])

In [ ]:
fig= go.Figure()

for i in range(10):
    fig.add_trace(go.Scatter(x=df[mae_cols].columns, y =df[mae_cols].values[i],
                             mode='lines+markers', name=titles[i]))

fig.show()

In [ ]:
import plotly.graph_objects as go

fig= go.Figure()

for i in range(10):
    fig.add_trace(go.Scatter(x=df[mse_cols].columns, y =df[mse_cols].values[i],
                             mode='lines+markers', name=titles[i]))

fig.show()